#### Test notebook that generates .tif files and stores in /results folder. Use **python37** kernel.

In [1]:
import os
import os.path
import sys
import rasterio
import pyproj
import zipfile
import tempfile
import random
import requests
import time

from rasterio.windows import Window
from rasterio import Affine
from shapely.geometry import Polygon
from shapely.ops import transform
from pathlib import Path
from datetime import datetime

from sentinel2download.downloader import Sentinel2Downloader
from sentinel2download.overlap import Sentinel2Overlap

# added by backend notebook_executor.py script:
REQUEST_ID = 8
AOI = 'POLYGON ((36.33779525756836 50.05697345978498, 36.3123893737793 50.05592644394527, 36.30964279174805 50.04517943449594, 36.31367683410645 50.03520185144665, 36.33110046386719 50.03035012265608, 36.353759765625 50.031122021389, 36.36208534240722 50.04176193236391, 36.35839462280273 50.052675038602, 36.35135650634766 50.05724898647173, 36.33779525756836 50.05697345978498))'
START_DATE = None
END_DATE = None

In [2]:
sys.version

'3.7.9 | packaged by conda-forge | (default, Dec  9 2020, 21:08:20) \n[GCC 9.3.0]'

In [3]:
NB_USER = os.getenv('NB_USER')
print(NB_USER)

jovyan


In [4]:
api_key = f"/home/{NB_USER}/work/.secret/sentinel2_google_api_key.json"

In [5]:
tetleha = [(
              36.63013458251953,
              49.87273390600898
            ),
            (
              36.81964874267578,
              49.87273390600898
            ),
            (
              36.81964874267578,
              49.98103318054302
            ),
            (
              36.63013458251953,
              49.98103318054302
            ),
            (
              36.63013458251953,
              49.87273390600898
            ), ]

forest = [(
              36.747894287109375,
              50.008400954338185
            ),
            (
              36.88556671142578,
              49.874282751903394
            ),
            (
              37.00366973876953,
              49.92647191927194
            ),
            (
              36.84711456298828,
              50.03707625174409
            ),
            (
              36.747894287109375,
              50.008400954338185
            ),
          ]
kharkiv = [(
              36.066741943359375,
              50.03773778732018
            ),
            (
              36.25556945800781,
              49.87295517274949
            ),
            (
              36.46568298339844,
              49.96447253142292
            ),
            (
              36.31874084472656,
              50.072565780723615
            ),
            (
              36.066741943359375,
              50.03773778732018
            ),
          ]

In [6]:
def _str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start="2020-01-01T00:00:00", end=datetime.today().strftime('%Y-%m-%dT%H:%M:%S')):
    return _str_time_prop(start, end, '%Y-%m-%dT%H:%M:%S', random.random())

In [7]:
def crop(input_path, output_path, polygon, name):
    with rasterio.open(input_path) as src:
        x_min, y_min, x_max, y_max = polygon.bounds
        
        affine = Affine(src.transform[0],
                        src.transform[1],
                        x_min,
                        src.transform[3],
                        src.transform[4],
                        y_max)

        row_min, col_max = rasterio.transform.rowcol(src.transform, x_max, y_max)
        row_max, col_min = rasterio.transform.rowcol(src.transform, x_min, y_min, op=round, precision=6)
        row_min = row_min + 1
        row_max = row_max + 1
        
        write_window = Window.from_slices([row_min, row_max], [col_min, col_max])
        raster = src.read(window=write_window)
        kwargs = src.meta.copy()
        kwargs.update({
            'height': write_window.height,
            'width': write_window.width,
            "transform": affine,
            'driver': 'GTiff'
        })

        with rasterio.open(output_path, 'w', **kwargs) as dst:
            if random.choice([True, False]):
                print(f"Randomly selected to add name {name}")
                dst.update_tags(name=name)
            if random.choice([True, False]):             
                start_date = random_date()
                end_date = random_date(start_date)
                print(f"Randomly selected to add start_date: {start_date} and end_date: {end_date}")   
                dst.update_tags(start_date=start_date, end_date=end_date)    
            # if random.choice([True, False]):
            #    request_id = 2
            #    print(f"Randomly selected to add request_id {request_id}")
            #    dst.update_tags(request_id=request_id)
            dst.write(raster)

In [8]:
def convert(input_path, output_path, format='GTiff'):
    with rasterio.open(input_path) as src:
        raster = src.read()
        crs = str(src.crs)
                
        kwargs = src.meta.copy()
        kwargs.update({'driver': format})
        
        
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            dst.write(raster)
        return crs

In [9]:
def to_crs(poly, target, current='EPSG:4326'):
    project = pyproj.Transformer.from_crs(pyproj.CRS(current), pyproj.CRS(target), always_xy=True).transform
    transformed_poly = transform(project, poly)
    return transformed_poly 

In [10]:
def load_tile(dir_):
    verbose = False
    tiles = {'36UYA', }
    product_type = 'L2A' 
    start_date = "2020-07-01"
    end_date = "2020-07-10"
    output_dir = dir_
    cores = 2
    BANDS = {'TCI',}
    CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 15.0, 'CLOUDY_PIXEL_PERCENTAGE': 10.0, }
    
    loader = Sentinel2Downloader(api_key, verbose=verbose)
    loaded = loader.download(product_type,
                             tiles,
                             start_date=start_date,
                             end_date=end_date,
                             output_dir=output_dir,
                             cores=cores,
                             bands=BANDS,
                             constraints=CONSTRAINTS)
    for save_path, _ in loaded:
        if save_path.endswith('TCI_10m.jp2'):
            print(save_path)
            return save_path

In [11]:
def generate(dir_, tile_path, aoi, location_name):
    
    stem = Path(tile_path).stem
    tif_path = f"{dir_}/{stem}.tif"    
    target_crs = convert(tile_path, tif_path)
    aoi = to_crs(Polygon(aoi), target_crs)

    temp_output_path = f"/home/{NB_USER}/work/results/example/{stem}_{location_name}.tif.temp"
    os.makedirs(os.path.dirname(temp_output_path), exist_ok=True)
    
    crop(tif_path, temp_output_path, aoi, location_name)
    
    print(f"Generated temp file: {temp_output_path}")
    output_path = temp_output_path[:-5]
    print(f"Rename temp file: {temp_output_path} to {output_path}")
    os.rename(temp_output_path, output_path)

In [12]:
dir_ = f"/home/{NB_USER}/work/satellite_imagery"

### main execution cell

In [13]:
tile_path = load_tile(dir_)
with tempfile.TemporaryDirectory(dir=dir_) as tmpdirname:
    print('Сreated temporary directory for calculations:', tmpdirname)
        
    locations = {'tetleha': tetleha, 'forest': forest, 'kharkiv': kharkiv}
    name = random.choice(list(locations.keys()))
    aoi = locations[name]
    
    generate(tmpdirname, tile_path, aoi, name)

print("Execution finished")

DefaultCredentialsError: File /home/jovyan/work/.secret/sentinel2_google_api_key.json was not found.